In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv('../input/airbnb-recruiting-new-user-bookings/train_users_2.csv.zip')
test = pd.read_csv('../input/airbnb-recruiting-new-user-bookings/test_users.csv.zip')

In [ ]:
submissionid=test.id

---> Test datam ve Train datamı birleştirdim. Burada Train datasına, Train datasının ortamalası vs ile yapacağım imputation'ları da tek seferde ekleyebilmek istediğim bu adımı gerçekleştirdim.

In [ ]:

# For identification purposes
train.loc[:,'Train'] = 1
test.loc[:,'Train'] = 0

test['country_destination'] = 0

df = pd.concat([train,test], ignore_index=True)

In [ ]:
train=df[df['Train']==1]

In [ ]:
test=df[df['Train']==0]

In [ ]:
train['age'].fillna(train['age'].mean(), inplace=True)

In [ ]:
test['age'].fillna(train['age'].mean(), inplace=True)

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
train['first_affiliate_tracked'] = train['first_affiliate_tracked'].fillna('Unknown')
test['first_affiliate_tracked'] = test['first_affiliate_tracked'].fillna('Unknown')

In [ ]:
train.drop(['date_account_created','timestamp_first_active','date_first_booking','Train'],axis=1,inplace=True)

In [ ]:
test.drop(['date_account_created','timestamp_first_active','date_first_booking','country_destination','Train'],axis=1,inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder 


In [ ]:
le = LabelEncoder() 
  
train['gender']= le.fit_transform(train['gender'])
train['signup_method']= le.fit_transform(train['signup_method']) 
train['first_affiliate_tracked']= le.fit_transform(train['first_affiliate_tracked']) 
train['signup_method']= le.fit_transform(train['signup_method']) 
train['language']= le.fit_transform(train['language'])
train['affiliate_channel']= le.fit_transform(train['affiliate_channel'])
train['affiliate_provider']= le.fit_transform(train['affiliate_provider'])
train['signup_app']= le.fit_transform(train['signup_app'])
train['first_device_type']= le.fit_transform(train['first_device_type'])
train['first_browser']= le.fit_transform(train['first_browser'])


In [ ]:
le = LabelEncoder() 
  
test['gender']= le.fit_transform(test['gender'])
test['signup_method']= le.fit_transform(test['signup_method']) 
test['first_affiliate_tracked']= le.fit_transform(test['first_affiliate_tracked']) 
test['signup_method']= le.fit_transform(test['signup_method']) 
test['language']= le.fit_transform(test['language'])
test['affiliate_channel']= le.fit_transform(test['affiliate_channel'])
test['affiliate_provider']= le.fit_transform(test['affiliate_provider'])
test['signup_app']= le.fit_transform(test['signup_app'])
test['first_device_type']= le.fit_transform(test['first_device_type'])
test['first_browser']= le.fit_transform(test['first_browser'])


In [ ]:
train.country_destination.replace('NDF',0,inplace=True)
train.country_destination.replace('US',1,inplace=True)
train.country_destination.replace('other',2,inplace=True)
train.country_destination.replace('FR',3,inplace=True)
train.country_destination.replace('CA',4,inplace=True)
train.country_destination.replace('GB',5,inplace=True)
train.country_destination.replace('ES',6,inplace=True)
train.country_destination.replace('IT',7,inplace=True)
train.country_destination.replace('PT',8,inplace=True)
train.country_destination.replace('NL',9,inplace=True)
train.country_destination.replace('DE',10,inplace=True)
train.country_destination.replace('AU',11,inplace=True)

In [ ]:
test

In [ ]:
from sklearn.model_selection import train_test_split
y=train['country_destination']
X=train.drop(['country_destination','id'],axis=1)
# split the dataset into train and test sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1,shuffle=True,stratify=y )

In [ ]:
pred_country={0:"NDF", 1:"US", 2:"other", 3:"FR", 4:"CA", 5:"GB", 6:"ES", 7:"IT", 8:"PT", 9:"DE", 10:"NL", 11:"AU"}

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

from sklearn.metrics import classification_report
target_names = ['NDF', 'US', 'other', 'FR', 'CA', 'GB', 'ES', 'IT', 'PT', 'NL','DE', 'AU']

rf=RandomForestClassifier(n_estimators=10)

#Train the model using the training sets y_pred=rf.predict(X_test)
rf.fit(X_train,y_train)

# prediction on test set
y_predrf=rf.predict(X_test)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print(classification_report(y_test, y_predrf, target_names=target_names))

In [ ]:
#predictionsrf=rf.predict(test.drop(['id'],axis=1))

In [ ]:
#results=[]
#for i in predictionsrf:
#    results.append(pred_country[i])
#print(results)

In [ ]:
#my_submissionrf = pd.DataFrame({'id': test.id, 'country':results})
#my_submissionrf.to_csv('submission.csv', index=False)

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {'solver': ['lbfgs'], 'max_iter': [1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000 ], 'alpha': 10.0 ** -np.arange(1, 10), 'hidden_layer_sizes':np.arange(10, 15), 'random_state':[0,1,2,3,4,5,6,7,8,9]}
mlpgridsearch = GridSearchCV(MLPClassifier(), parameters, n_jobs=-1)
mlpgridsearch.fit(X_train,y_train)
predsgridmlp = mlpgridsearch.predict(X_test)

print(mlpgridsearch.score(X_train, y_train))
print(mlpgridsearch.best_params_)

In [ ]:
from sklearn.neural_network import MLPClassifier
#Generate prediction using Neural Net

#mlp = MLPClassifier()
#mlp.fit(X_train,y_train)
#predsmlp = mlp.predict(X_test)
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print(classification_report(y_test, predsgridmlp, target_names=target_names))

In [ ]:
predictionsmlp=mlpgridsearch.predict(test.drop(['id'],axis=1))

In [ ]:
results=[]
for i in predictionmlp:
    results.append(pred_country[i])
print(results)

In [ ]:
my_submissionmlp = pd.DataFrame({'id': test.id, 'country':results})
my_submissionmlp.to_csv('submission.csv', index=False)

import xgboost
from xgboost import XGBClassifier
xgb = XGBClassifier(max_depth=6, learning_rate=0.3, n_estimators=25,
                    objective='multi:softprob', subsample=0.5, colsample_bytree=0.5, seed=0)                  
xgb.fit(X_train, y_train)
y_pred = xgb.predict_proba(X_test)
print(classification_report(y_test, y_predrf, target_names=target_names))

In [ ]:
#predictionsxgb=xgb.predict(test.drop(['id'],axis=1))

Gridsearch yapıp xgboost denenebilir veya bir önceki drafta nn kullanarak yaptıgın modele gridsearch denenebilir. Zaten hepsinden iyi çalıştı. başka boosting yöntemleri denenebilir. Yeni özellik eklenmeye çalışılabilir. Seasonal bir etki var çünkü.

In [ ]:
#results=[]
#for i in predictionsxgb:
#    results.append(pred_country[i])
#print(results)

In [ ]:
#my_submissionxgb = pd.DataFrame({'id': test.id, 'country':results})
#my_submissionxgb.to_csv('submission.csv', index=False)

MLP classifiyer en iyi sonucu verdi şimdiye kadar. Ama rf ve xgb scoru arasında pek bir fark oldugu söylenemez. 0.68 0.69 alabildiğim en yüksek score. Grid searchle modelimi geliştirecek en uygun parametreleri seçmeyi deneyebilirim.